In [1]:
import scipy.sparse as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
h = 200 
w =  200
ds = 0.015
sigma_l_i    = 1.74                  # (mS/cm)
sigma_t_i    = 0.19                  # (mS/cm)
sigma_l_e    = 6.25                  # (mS/cm)
sigma_t_e    = 2.36                  # (mS/cm)

xx =  sigma_l_i
yy =  sigma_t_i

In [3]:
def ivec_real( y, x ):
    _x = x if x >=0 else w+x 
    _y = y if y >=0 else h+y 
    return w*_y + _x

def ivec_all( y, x ):
    _x = x if x >=0 else (w+2)+x 
    _y = y if y >=0 else (h+2)+y 
    return (w+2)*_y + _x

In [4]:
size_real = h*w
size_all  = (h+2)*(w+2)

In [5]:
# Isolated Boundary condition : A1(size_real, size_all)
A1 = sp.lil_matrix((size_all, size_real), dtype=np.float32)
#A1 = np.zeros((size_all, size_real), dtype=np.float64)

# - Real points
for n in range(h):
    for m in range(w):
          A1[ ivec_all(n+1, m+1), ivec_real(n,m)] = 1.0

# - 4 corners
A1[ ivec_all( 0, 0), ivec_real( 0, 0)] =  2.0 
A1[ ivec_all( 0, 0), ivec_real( 1, 0)] = -2.0 
A1[ ivec_all( 0, 0), ivec_real( 0, 1)] = -2.0
A1[ ivec_all( 0, 0), ivec_real( 1, 1)] =  3.0
A1[ ivec_all( 1, 0), ivec_real( 0, 0)] =  1.0
A1[ ivec_all( 1, 0), ivec_real( 1, 0)] = -1.0
A1[ ivec_all( 1, 0), ivec_real( 1, 1)] =  1.0
A1[ ivec_all( 0, 1), ivec_real( 0, 0)] =  1.0
A1[ ivec_all( 0, 1), ivec_real( 0, 1)] = -1.0
A1[ ivec_all( 0, 1), ivec_real( 1, 1)] =  1.0

A1[ ivec_all( -1, 0), ivec_real( -1, 0)] =  2.0 
A1[ ivec_all( -1, 0), ivec_real( -2, 0)] = -2.0 
A1[ ivec_all( -1, 0), ivec_real( -1, 1)] = -2.0
A1[ ivec_all( -1, 0), ivec_real( -2, 1)] =  3.0
A1[ ivec_all( -2, 0), ivec_real( -1, 0)] =  1.0
A1[ ivec_all( -2, 0), ivec_real( -2, 0)] = -1.0
A1[ ivec_all( -2, 0), ivec_real( -2, 1)] =  1.0
A1[ ivec_all( -1, 1), ivec_real( -1, 0)] =  1.0
A1[ ivec_all( -1, 1), ivec_real( -1, 1)] = -1.0
A1[ ivec_all( -1, 1), ivec_real( -2, 1)] =  1.0

A1[ ivec_all( 0, -1), ivec_real( 0, -1)] =  2.0 
A1[ ivec_all( 0, -1), ivec_real( 1, -1)] = -2.0 
A1[ ivec_all( 0, -1), ivec_real( 0, -2)] = -2.0
A1[ ivec_all( 0, -1), ivec_real( 1, -2)] =  3.0
A1[ ivec_all( 1, -1), ivec_real( 0, -1)] =  1.0
A1[ ivec_all( 1, -1), ivec_real( 1, -1)] = -1.0
A1[ ivec_all( 1, -1), ivec_real( 1, -2)] =  1.0
A1[ ivec_all( 0, -2), ivec_real( 0, -1)] =  1.0
A1[ ivec_all( 0, -2), ivec_real( 0, -2)] = -1.0
A1[ ivec_all( 0, -2), ivec_real( 1, -2)] =  1.0

A1[ ivec_all( -1, -1), ivec_real( -1, -1)] =  2.0 
A1[ ivec_all( -1, -1), ivec_real( -2, -1)] = -2.0 
A1[ ivec_all( -1, -1), ivec_real( -1, -2)] = -2.0
A1[ ivec_all( -1, -1), ivec_real( -2, -2)] =  3.0
A1[ ivec_all( -2, -1), ivec_real( -1, -1)] =  1.0
A1[ ivec_all( -2, -1), ivec_real( -2, -1)] = -1.0
A1[ ivec_all( -2, -1), ivec_real( -2, -2)] =  1.0
A1[ ivec_all( -1, -2), ivec_real( -1, -1)] =  1.0
A1[ ivec_all( -1, -2), ivec_real( -1, -2)] = -1.0
A1[ ivec_all( -1, -2), ivec_real( -2, -2)] =  1.0

# - 4 edges
for m in range(1,w-1):
    A1[ ivec_all(  0, m+1), ivec_real(  0, m)] =  1.0
    A1[ ivec_all( -1, m+1), ivec_real( -1, m)] =  1.0

for n in range(1,h-1):
    A1[ ivec_all( n+1,  0), ivec_real( n,  0)] =  1.0
    A1[ ivec_all( n+1, -1), ivec_real( n, -1)] =  1.0

print A1.shape

(40804, 40000)


In [6]:
# Parabolic PDE coefficients
#A2 = np.zeros((size_real, size_all), dtype=np.float32)
A2 = sp.lil_matrix((size_real, size_all), dtype=np.float32)

for y in range(h):
    for x in range(w):
        A2[ ivec_real(y,x), ivec_all(y+1, x+1)] = -(2*xx+2*yy)/(ds**2)
        A2[ ivec_real(y,x), ivec_all(y  , x+1)] = yy/(ds**2)
        A2[ ivec_real(y,x), ivec_all(y+2, x+1)] = yy/(ds**2)
        A2[ ivec_real(y,x), ivec_all(y+1, x  )] = xx/(ds**2)
        A2[ ivec_real(y,x), ivec_all(y+1, x+2)] = xx/(ds**2)

print A2.shape

(40000, 40804)


In [7]:
A1_sp =sp.csr_matrix(A1)
A2_sp =sp.csr_matrix(A2)
A = A2_sp.dot(A1_sp)

In [8]:
A_ = A.todense()
Dinv = (1/np.diag(A_))[:, np.newaxis]
R = A_ - np.diag(np.diag(A_))

In [34]:
A_diag = A.diagonal()
Dinv = (1/A_diag)[:, np.newaxis]
R = A.todense() - np.diag(A_diag)

In [30]:
#print np.diag(A_)
print np.diag(np.diag(A_))
#print A.diagonal()
print np.diag( A.diagonal() )

[[ -8577.77636719      0.              0.         ...,      0.              0.
       0.        ]
 [     0.         -16311.11035156      0.         ...,      0.              0.
       0.        ]
 [     0.              0.         -16311.11035156 ...,      0.              0.
       0.        ]
 ..., 
 [     0.              0.              0.         ..., -16311.11035156
       0.              0.        ]
 [     0.              0.              0.         ...,      0.
  -16311.11035156      0.        ]
 [     0.              0.              0.         ...,      0.              0.
   -8577.77636719]]
[[ -8577.77636719      0.              0.         ...,      0.              0.
       0.        ]
 [     0.         -16311.11035156      0.         ...,      0.              0.
       0.        ]
 [     0.              0.         -16311.11035156 ...,      0.              0.
       0.        ]
 ..., 
 [     0.              0.              0.         ..., -16311.11035156
       0.              0

In [10]:
print Dinv.shape
print R.shape

(40000, 1)
(40000, 40000)


In [11]:
import chainer
from chainer import cuda
import cupy as xp
cuda.get_device(0).use()

In [12]:
A_xp =cuda.to_gpu(A_, device=0)
R_xp = cuda.to_gpu(R, device=0)
#A2_xp =cuda.to_gpu(A2, device=0)

CUDARuntimeError: cudaErrorMemoryAllocation: out of memory